In [5]:
# This notebook demonstrates gradient descent methods with fixed step size and backtracking line search.
# It was prepared by Vlad Kobzar (vk283@nyu.edu) for the Machine Learning course at NYU's Center for 
# Data Science
# https://davidrosenberg.github.io/ml2017/
# February 5, 2017

# To install matplotlib, use e.g. pip install matplotlib

# To display animations, we use ffmpeg. To check whether this package has been installed on your
# computer run: 
#      import matplotlib.animation as animation
#      print(animation.writers.list())
# If that shows ffmpeg, they installed it. 
#
# Otherwise, it can be installed using, e.g., homebrew. 
#     brew install ffmpeg
# To install homebrew on MAC, run
#     /usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"

In [6]:
import numpy as np
import sympy as sp
from numpy import linalg

# evaluate_gradient returns a numpy array representing the gradient 
# of a function evaluated at a point.  
def evaluate_gradient(f_str, point, variables=['x', 'y']):
    var = []
    grad = []
    for i in variables:
        var.append(sp.symbols(i))
    if len(point) != len(variables):
        raise ValueError('Length of points doesnt match the number of variables')
    for i in range(len(var)):
        diff_func = sp.lambdify(variables, sp.diff(sp.sympify(f_str), var[i]))
        grad.append(diff_func(*point))
    return np.array(grad)

# fixed_step_gd returns a list of numpy 2D arrays representing 
# the steps of a gradient descent minimization algorithm with 
# a fixed step size with respect to a function of two variables. 
def fixed_step_gd(f_str, initial_vector, step_size, 
                              maxstep = 12, precision=0.2):
    x=initial_vector
    steps=[x]
    grad = np.array ([0, 2*precision]) 
    i=0
    while linalg.norm(grad,2)> precision and (i< maxstep):
        grad=-evaluate_gradient(f_str, [x[0],x[1]])
        x = x +step_size*grad
        i+=1
        steps.append(x) 
    return steps

# backtracking_line_search returns a list of numpy 2D arrays 
# representing the steps of a gradient descent minimization 
# with a backtracking line search for a function of two variables.  
# The parameters alpha must be in (0...0.5) and beta in (0...1). 
# See, e.g., Boyd, Algorithm 9.2.
def backtracking_line_search_gd(f_str, initial_vector,
                              maxstep = 12, precision=0.2, alpha = 0.1, beta =0.7):
    x = initial_vector
    steps=[initial_vector] 
    grad=np.array([0, 2*precision]) 
    i=0
    f_lambda = sp.lambdify(['x', 'y'], f_str, "numpy")
    #This loop calculates the coordinates of each step.
    while (linalg.norm(grad,2) > precision) and (i< maxstep):
        grad=-evaluate_gradient(func_str, [x[0],x[1]])
        t=1
        i+=1
        j=0
        funcnextstep=f_lambda(x[0]+ t*grad[0],x[1]+ t*grad[1])
        linearextrapolation = f_lambda(x[0], x[1])+alpha*t*np.dot(evaluate_gradient(f_str, x), grad)
        #This loop calculates the size of each step 
        while (funcnextstep> linearextrapolation) and (i< maxbacktrackstep):
            t=beta*t
            j += 1
            funcnextstep=f_lambda(x[0]+ t*grad[0],x[1]+ t*grad[1]) 
            linearextrapolation = f_lambda(x[0], x[1])+alpha*t*np.dot(evaluate_gradient(f_str, x), grad)
        x=x+t*grad
        steps.append(x)   
    return steps

In [7]:
# init and animate are called by animation at initialization 
# and each frame of the animation
def init():
    global  points, backtracking_points
    pts.set_data ([],[])
    sct.set_data([],[])
    backtracking_pts.set_data ([],[])
    backtracking_sct.set_data([],[])
    return pts, sct, backtracking_pts, backtracking_sct

def animate(i):
    global  points, backtracking_points
    if i <len (points):
        pts.set_data([item[0] for item in points[0:i]], [item[1] for item in points[0:i]])
        sct.set_data([item[0] for item in points[0:(i)]], [item[1] for item in points[0:(i)]])
    if i <len (backtracking_points):    
        backtracking_pts.set_data([item[0] for item in backtracking_points[0:i]], 
                                    [item[1] for item in backtracking_points[0:i]]) 
        backtracking_sct.set_data([item[0] for item in backtracking_points[0:(i)]],
                                [item[1] for item in backtracking_points[0:(i)]])
    return pts, sct, backtracking_pts, backtracking_sct

In [8]:
# The following parameters are set by the user
x_init=np.array([-1,1]) # initial search point
xmin = x_init[0]-.5     # boundaries of the displayed domain
xmax = x_init[0]+4      # +1.5 Use the commented constants are 
ymin = x_init[1]-.5     # -2   used to generate 
ymax = x_init[1]+4      # +.5  figure 9.3 from Boyd & Vandenberge, Convex Optimization
grid_size = 40
precision = 0.2         #gradient l2 norm threshold for the stopping condition

func_str = '3*(.4*x)**2 - (.4*x)*y +0.5*y**2-(.4*x)-3*y' #The objective function (of x and y)

# Use the below to generate figure 9.3 from Boyd
# func_str =  'exp(x+3*y-0.1)+ exp(x-3*y-0.1)+exp(-x-0.1)'#Fixed GD step size
# Fixed step search doesn't work, for example, for the Boyd example. To turn it off, 
# maxgdstep should be set to 1.

gdstep = 0.5            # Size of a fixed step
maxbacktrackstep=25     # Maximum number of steps for the backtracking line search 
maxgdstep=25            # Maximum number of steps for the fixed step search 

# The following code generates the animation
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
rc('animation', html='html5') 

# The following code generates the search steps and countour lines
func_sym=sp.sympify(func_str)
func_lambda = sp.lambdify(['x', 'y'], func_str, "numpy")
points = fixed_step_gd (func_str, x_init, gdstep, maxgdstep, precision)
backtracking_points= backtracking_line_search_gd(func_str, x_init, maxbacktrackstep,  precision)
x = np.linspace(xmin, xmax, grid_size)
y = np.linspace(ymin, ymax, grid_size)
xx, yy = np.meshgrid(x, y)
z = func_lambda(x[None,:],y[:,None] )
levels = np.array(sorted([func_lambda(p[0],p[1]) for p in backtracking_points]))

# The following code plots the search steps and countour lines
fig = plt.figure()
ax = fig.add_subplot(111)
plt.gca().set_aspect('equal', adjustable='box')
cp = plt.contour(xx, yy,z, levels = levels)
plt.title('Gradient Descent \n f(x,y)=%s' %func_str, fontsize=7)
plt.xlabel('x ')
plt.ylabel('y ') 

pts, = ax.plot([],[], color='red', 
               label='Fixed step size:  %.2f \n in %d steps' % (gdstep, (len(points)-1)))
sct,  = ax.plot([], 'ro', markersize=3)
backtracking_pts, = ax.plot([],[], color='black', 
                            label='Backtracking line search\n in %d steps' % (len(backtracking_points)-1))
backtracking_sct, = ax.plot([], 'ko', markersize=3)
plt.legend([pts,backtracking_pts], [pts.get_label(), backtracking_pts.get_label()], loc='best', fontsize=7)

anim = animation.FuncAnimation(fig, animate, frames= max([len(points),
                               len (backtracking_points)]), interval=500, blit=True, init_func=init)
HTML(anim.to_html5_video())